I write same code for practice with reference to [PHALANX's notebook](https://www.kaggle.com/phalanx/train-swin-t-pytorch-lightning)  . PHALANX used Swin-Transfomer model.  Maybe you should see PHALANX's notebook than this notebook.

In [ ]:
!pip install python-box timm pytorch-lightning==1.4.0 grad-cam ttach

In [ ]:
import os
import warnings
from pprint import pprint
from glob import glob
from tqdm import tqdm

import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms as T
from box import Box
from timm import create_model
from sklearn.model_selection import StratifiedKFold
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
from pytorch_grad_cam import GradCAMPlusPlus
from pytorch_grad_cam.utils.image import show_cam_on_image

import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import LightningDataModule, LightningModule

warnings.filterwarnings("ignore")

## config

In [ ]:
config = {'seed': 2021,
          'root': '/kaggle/input/petfinder-pawpularity-score/', 
          'n_splits': 5,
          'epoch': 20,
          'trainer': {
              'gpus': 1,
              'accumulate_grad_batches': 1,
              'progress_bar_refresh_rate': 1,
              'fast_dev_run': False,
              'num_sanity_val_steps': 0,
              'resume_from_checkpoint': None,
          },
          'transform':{
              'name': 'get_default_transforms',
              'image_size': 224
          },
          'train_loader':{
              'batch_size': 64,
              'shuffle': True,
              'num_workers': 4,
              'pin_memory': False,
              'drop_last': True,
          },
          'val_loader': {
              'batch_size': 64,
              'shuffle': False,
              'num_workers': 4,
              'pin_memory': False,
              'drop_last': False
         },
          'test_loader' : {
              'batch_size' : 64,
              'shuffle' : False,
              'num_workers' : 4,
              'pin_memory' : False,
              'drop_last' : False
          },
          'model':{
              'name': 'swin_tiny_patch4_window7_224',
              'output_dim': 1
          },
          'optimizer':{
              'name': 'optim.AdamW',
              'params':{
                  'lr': 1e-5
              },
          },
          'scheduler':{
              'name': 'optim.lr_scheduler.CosineAnnealingWarmRestarts',
              'params':{
                  'T_0': 20,
                  'eta_min': 1e-4,
              }
          },
          'loss': 'nn.BCEWithLogitsLoss',
}

config = Box(config)
pprint(config)

## dataset

torchvisioin

PyTorchと連携するコンピュータービジョン用のライブラリです。効率的な画像およびビデオ変換用のユーティリティ、一般的に使用される事前トレーニング済みモデル、および一部のデータセットtorchvisionがあります

1.transformsによる前処理の定義、2.Datasetsによる前処理&ダウンロード　3.DataloaderによるDatasetの使用という流れになる


Dataset
データセットを作成する

1.transformsによる前処理の定義、2.Datasetsによる前処理&ダウンロード　3.DataloaderによるDatasetの使用という流れになる

DataLoader

Dataset からサンプルを取得して、ミニバッチを作成するクラスです。基本的には、サンプルを取得する Dataset とバッチサイズを指定して作成します。

In [ ]:
class PetfinderDataset(Dataset):
    """画像:image
       label:pawpularity
       の二つだけが含まれたデータセットを作る
    """
    def __init__(self, df, image_size=224):
        self._X = df["Id"].values
        self._y = None
        if "Pawpularity" in df.keys():
            self._y = df["Pawpularity"].values
        self._transform = T.Resize([image_size, image_size])#torchvision.transforms
    
    def __len__(self):
        """データ数を返す"""
        return len(self._X)
    
    def __getitem__(self, idx):
        """pawpularityがあれば、pawpularity と　画像を返す
            pawpularityがなければ、画像だけ返す
        """
        image_path = self._X[idx]
        image = read_image(image_path)#torchvision.io
        image = self._transform(image)
        if self._y is not None:
            label = self._y[idx]# insert pawpulariy
            return image, label
        return image
    

class PetfinderDataModule(LightningDataModule):
    def __init__(
        self,
        train_df,
        val_df,
        cfg,
    ):
        super().__init__()
        self._train_df = train_df
        self._val_df = val_df
        self._cfg = cfg
        
    def __create_dataset(self, train=True):
        """訓練データか検証データか判定して、データセットを作成"""
        return(
            PetfinderDataset(self._train_df, self._cfg.transform.image_size)
            if train#後置if
            else PetfinderDataset(self._val_df, self._cfg.transform.image_size)
        )
    
    def train_dataloader(self):
        dataset = self.__create_dataset(True)
        #{'batch_size': 64,
        #'drop_last': False,
        #'num_workers': 4,
        #'pin_memory': False,
        #'shuffle': False}}
        return DataLoader(dataset, **self._cfg.train_loader)#configのval_loaderキーの値を渡す
    
    def val_dataloader(self):
        dataset = self.__create_dataset(False)
        return DataLoader(dataset, **self._cfg.val_loader)
    
class PetfindertestDataModule(LightningDataModule):
    def __init__(self, test_df, cfg):
        super().__init__()
        self._test_df = test_df
        self._cfg = cfg
        
    def __create_test_dataset(self):
        """テストデータ用のデータセット作成"""
        return PetfinderDataset(self._test_df, self._cfg.transform.image_size)
    
    def test_dataloader(self):
        dataset = self.__create_test_dataset()
        return DataLoader(dataset, **self._cfg.test_loader)

## visualize data

seed_everything

Function that sets seed for pseudo-random number generators in: pytorch, numpy, python.random In addition, sets the following environment 
variables:

utograd.set_detect_anomaly

Context-manager that enable anomaly detection for the autograd engine.

This does two things:

Running the forward pass with detection enabled will allow the backward pass to print the traceback of the forward operation that created the failing backward function.

Any backward computation that generate “nan” value will raise an error.

anomaly detetionは異常検知の意味



In [ ]:
torch.autograd.set_detect_anomaly(True)
seed_everything(config.seed)#seed_everything(config.seed)



df = pd.read_csv(os.path.join(config.root, "train.csv"))
#configファイル 'root': '/kaggle/input/petfinder-pawpularity-score/'
df["Id"] = df["Id"].apply(lambda x: os.path.join(config.root, "train", x + ".jpg"))

In [ ]:
df["Id"][0]

In [ ]:
sample_dataloader = PetfinderDataModule(df, df, config).val_dataloader()
images, labels = iter(sample_dataloader).next()#iterとnextを組み合わせることで、最初の要素からnextを実行するたびに順番に取り出すことができる

plt.figure(figsize = (12, 12))
for it, (image, label) in enumerate(zip(images[:16], labels[:16])):
    plt.subplot(4, 4, it + 1)
    plt.imshow(image.permute(1, 2, 0))#指定した次元に行列を変換
    plt.axis("off")
    plt.title(f'Pawpularity : {int(label)}')

## augmentation

compose
指定した変換を上から順番に一つずつやっていく


concertImageDtype

Convert a tensor image to the given dtype and scale the values accordingly This function does not support PIL Image.

Normalize

正規化を行う Transform です。 nn 個のチャンネルごとの平均 (m_1, m_2, \cdots, m_n)(m 
1
​
 ,m 
2
​
 ,⋯,m 
n
​
 ) 及び標準偏差 (s_1, s_2, \cdots, s_n)(s 
1
​
 ,s 
2
​
 ,⋯,s 
n
​
 ) が与えられたとき、チャンネルごとに次のように標準化を行います。

output = (input-m)/s

アフィン変換
画像を行列を使って、拡大・縮小・回転・平行移動すること

In [ ]:
IMAGENET_MEAN = [0.485, 0.456, 0.406]#RGB
IMAGENET_STD = [0.229, 0.224, 0.225]#RGB

def get_default_transforms():
    transform = {
        "train" : T.Compose(
            [
                T.RandomHorizontalFlip(),#左右反転
                T.RandomVerticalFlip(),#上下反転
                T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),#ランダムにアフィン変換
                T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),#ランダムに明るさ、コントラスト、彩度、色相を変化させる
                T.ConvertImageDtype(torch.float),#torchのfloat型に写真データを変換
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),#正規化
            ]
        ),
        "val":T.Compose(
            [
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
        ),
    }
    
    return transform

## model

　Pythonのassert文は、基本的には、条件をテストするデバッグ支援ツールです。アサーションの条件がTrueの場合は何も起きず、プログラムは何事もなく動作し続けます。しかし、アサーションの条件がFalseと評価された場合はAssertionError例外が送出され、必要に応じてエラーメッセージが生成されます。
 
 assert 条件式, 条件式がFalseの場合に出力するメッセージ
 
 randperm
 Tensor0 から n - 1 までの整数のランダム順列を返します。
 randperm(4) →　[2, 1, 0, 3]
 
 lightningModule →　生PyTorchで書かなければならない学習ループやバリデーションループ等を各hookのメソッドとして整理したフレームワークです。他にもGPUの制御やコールバックといった処理もフレームワークに含み、可読性や学習の再現性を上げています。
                     torch.nn.Moduleの作成と似ていますが、ニューラルネットワークのmodelを定義するだけでなく、バッチに対するlossの計算、optimizerまで定義するクラスになっています。作成したモデルをTrainerクラスに渡してfitメソッドで学習を行います。

create_model:画像認識事前学習済みモデルライブラリpretrained→事前学習済みのモデルを使うか　num_classes→予測するクラス数 in_chain→学習の回数？連鎖率？

nn.Sequential →　各層をつなげるもの kerasのsequantialと同じ

lightningmodule 参考サイト

https://qiita.com/ground0state/items/c1d705ca2ee329cdfae4#pytorch_lightninglightningmodule

scheduler epoch毎に学習率を変化させてくれるもの
http://blog.livedoor.jp/dividend_tomtom/archives/9193475.html

In [ ]:
def mixup(x: torch.Tensor, y:torch.Tensor, alpha: float=1.0):
    assert alpha > 0, "alpha shoula be larger than 0"
    assert x.size(0) > 1, "Mixup cannot be applied to a single instance"
    
    lam = np.random.beta(alpha, alpha)#np.random.beta()は、パラメータa, bのベータ分布に従う乱数を返す
    rand_index = torch.randperm(x.size()[0])
    mixed_x = lam*x + (1 - lam) * x[rand_index, :]
    target_a, target_b = y, y[rand_index]
    return mixed_x, target_a, target_b, lam

class Model(pl.LightningModule):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.__build_model()
        self._criterion = eval(self.cfg.loss)()
        self.transform = get_default_transforms()
        self.save_hyperparameters(cfg)
        
        
    def __build_model(self):
        self.backbone = create_model(
            self.cfg.model.name, pretrained=True, num_classes=0, in_chans=3
        )#ニューラルネットワークのmodelをreturnします。
        num_features = self.backbone.num_features
        self.fc = nn.Sequential(
            nn.Dropout(0.5), nn.Linear(num_features, self.cfg.model.output_dim)#追加で学習させる層num_featureの正体は？ #Linearは直線
        )
    
    def forward(self, x):
        """順伝播処理"""
        f = self.backbone(x)
        out = self.fc(f)
        return out
    
    def training_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, "train")
        return {"loss" : loss, "pred" : pred, "labels" : labels}
    
    def validation_step(self, batch, batch_index):
        loss, pred, labels = self.__share_step(batch, "val")
        return {"pred" : pred, "labels" : labels}
    
    def __share_step(self, batch, mode):
        images, labels = batch
        labels = labels.float() / 100.0
        images = self.transform[mode](images)#modeに応じてaugmentationを実行
        
        if torch.rand(1)[0] < 0.5 and mode == "train":
            mix_images, target_a, target_b, lam = mixup(images, labels, alpha=0.5)
            logits = self.forward(mix_images).squeeze(1)
            loss = self._criterion(logits, target_a) * lam + (1 - lam)*self._criterion(logits, target_b)
        else:
            logits = self.forward(images).squeeze(1)#指定した次元（axis=1の次元を削除）
            loss = self._criterion(logits, labels)
            
        pred = logits.sigmoid().detach().cpu() * 100.#detach 同一デバイス上に新しいテンソルを作成、計算グラフから切り離される
        labels = labels.detach().cpu() * 100.
        return loss, pred, labels
    
    def trainig_epoch_end(self, outputs):
        """1エポック終わった後の処理"""
        self.__share_epoch_end(outputs, "train")
        
    def validation_epoch_end(self, outputs):
        """１エポック終わった後の処理"""
        self.__share_epoch_end(outputs, "val")
        
    def __share_epoch_end(self, outputs, mode):
        preds = []
        labels = []
        for out in outputs:
            pred, label = out["pred"], out["labels"]
            preds.append(pred)
            labels.append(label)
        preds = torch.cat(preds)
        labels= torch.cat(labels)
        metrics = torch.sqrt(((labels - preds) ** 2).mean())#RMSE
        self.log(f'{mode}_loss', metrics )
        
    def check_gradcam(self, dataloader, target_layer, target_category, reshape_transform=None):
        """画像のどこに注目しているのか描画する"""
        cam = GradCAMPlusPlus(
            model = self,
            #target_layer = [target_layer],#モデルごとに決まっている定数
            target_layers = [target_layer],
            use_cuda = self.cfg.trainer.gpus,#1
            reshape_transform = reshape_transform)
        org_images, labels = iter(dataloader).next()#orginalの画像とlbaleを取り出す
        cam.batch_size = len(org_images)
        images = self.transform["val"](org_images)#val用のaugmentation
        images = images.to(self.device)#GPUの切り替え
        logits = self.forward(images).squeeze(1)
        pred = logits.sigmoid().detach().cpu().numpy() * 100
        labels = labels.cpu().numpy()
        
        grayscale_cam = cam(input_tensor=images, target_category=target_category, eigen_smooth = True)#eigen_camをsmooth_gradで実行
        org_images = org_images.detach().cpu().numpy().transpose(0, 2, 3, 1) / 255.#transpose配列の順番の入れ替え

        return org_images, grayscale_cam, pred, labels
    
    def configure_optimizers(self):
        """optimizerをreturnする。
        schedulerを使用する場合はreturnをoptimizerのリストと
        schedulerのリストのタプルとする。"""
        optimizer = eval(self.cfg.optimizer.name)(#optim.AdamW
            self.parameters(), **self.cfg.optimizer.params#lr=1e-5
        )
        scheduler = eval(self.cfg.scheduler.name)(#optim.lr_sccheduler.CosineAnnealingWarmRestarts
            optimizer,
            **self.cfg.scheduler.params#T_0:2θ,eta_min:1e-4
        )
        return [optimizer], [scheduler]

In [ ]:
a = np.arange(9)
a = a.reshape(3, 3)
print(a)
b = np.zeros((3, 1, 1, 1))
print(b)

In [ ]:
b.squeeze(1)

## train

callbacks.LearningRateMonitor

Automatically monitor and logs learning rate for learning rate schedulers during training.

modelcheckpoint

Automatically save model checkpoints during training.

https://torch.classcat.com/2021/02/22/pytorch-lightning-1-1-notebooks-05-trainer-flags-overview-2/
https://pytorch-lightning.readthedocs.io/en/stable/api/pytorch_lightning.callbacks.model_checkpoint.html

TensorBoardLogger

Log to local file system in TensorBoard format.

http://maruo51.com/2020/02/17/pytorch_lightning_log/


In [ ]:
skf = StratifiedKFold(
    n_splits=config.n_splits, shuffle=True, random_state=config.seed
)

for fold, (train_idx, val_idx) in enumerate(skf.split(df["Id"], df["Pawpularity"])):
    train_df = df.loc[train_idx].reset_index(drop=True)
    val_df = df.loc[val_idx].reset_index(drop=True)
    datamodule = PetfinderDataModule(train_df, val_df, config)
    model = Model(config)
    earystopping = EarlyStopping(monitor="val_loss")
    lr_monitor = callbacks.LearningRateMonitor()
    loss_checkpoint = callbacks.ModelCheckpoint(
        filename="best_loss",
        monitor="val_loss",
        save_top_k=1,
        mode="min",
        save_last=False,
    )
    logger = TensorBoardLogger(config.model.name)
    
    trainer = pl.Trainer(
        logger=logger,
        max_epochs=config.epoch,
        callbacks=[lr_monitor, loss_checkpoint, earystopping],
        **config.trainer,
    )
    trainer.fit(model, datamodule=datamodule)

## class activation map

In [ ]:
# gradsam reshape_transform for vit

def reshape_transform(tensor, height=7, width = 7):
    result = tensor.reshape(tensor.size(0),
                           height, width, tensor.size(2))
    
    #like in CNNs.
    result = result.permute(0, 3, 1, 2)
    return result

In [ ]:
 #gradcam reshape_transform for vit
def reshape_transform(tensor, height=7, width=7):
    result = tensor.reshape(tensor.size(0),
                            height, width, tensor.size(2))

    # like in CNNs.
    result = result.permute(0, 3, 1, 2)
    return result

In [ ]:
model = Model(config) 
model.load_state_dict(torch.load(f'{config.model.name}/default/version_0/checkpoints/best_loss.ckpt')['state_dict'])
model = model.cuda().eval()
config.val_loader.batch_size = 16
datamodule = PetfinderDataModule(train_df, val_df, config)
images, grayscale_cams, preds, labels = model.check_gradcam(
                                            datamodule.val_dataloader(), 
                                            target_layer=model.backbone.layers[-1].blocks[-1].norm1,
                                            target_category=None,
                                            reshape_transform=reshape_transform)

In [ ]:
plt.figure(figsize = (12, 12))
for it, (image, grayscale_cam, pred, label) in enumerate(zip(images, grayscale_cams, preds, labels)):
    plt.subplot(4, 4, it + 1)
    visualization = show_cam_on_image(image, grayscale_cam)#heatmapをのせた画像を作成
    plt.imshow(visualization)
    plt.title(f'pred : {pred:.1f} label : {label}')
    plt.axis("off")

### visualize result

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator


path = glob(f'./{config.model.name}/default/version_0/events*')[0]
event_acc = EventAccumulator(path, size_guidance={"scalars" : 0})
event_acc.Reload()

scalars = {}
for tag in event_acc.Tags()["scalars"]:
    events = event_acc.Scalars(tag)
    scalars[tag] = [event.value for event in events]

In [ ]:
import seaborn as sns

sns.set()

plt.figure(figsize = (16, 6))
plt.subplot(1, 2, 1)
plt.plot(range(len(scalars["lr-AdamW"])), scalars["lr-AdamW"])
plt.xlabel("epoch")
plt.ylabel("lr")
plt.title("adamw lr")

plt.subplot(1, 2, 2)
#plt.plot(range(len(scalars['train_loss'])), scalars['train_loss'], label = 'train_loss')
plt.plot(range(len(scalars['val_loss'])), scalars['val_loss'], label = 'val_loss')
plt.legend()
plt.ylabel("rmse")
plt.xlabel("epoch")
plt.title("train/val rmse")
plt.show()

In [ ]:
print("best_val_loss", min(scalars["val_loss"]))